In [1]:
"""
canonical imports here
"""

import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import tensorflow_hub as hub
import numpy as np
import tensorflow_text
import datetime as dt
import re

# Data Cleanage (FR) aka Data Cleaning

## Import Data

In [2]:
#NOTE in the packages we eventually don't limit the rows to 5000
dftest = pd.read_json('/Users/ianndelap/code/ianndelap/market_sentiment/raw_data/dump.json',lines=True,nrows=7000)

"""
let's choose only the features we need
"""
df = dftest[['full_text',
'lang',
'datetimestamp',
'reply_count',
'retweet_count',
'quote_count',
'created_at',
'favorite_count',
'symbols',
'hashtags']]

## Let's cleanse this mother…!

In [3]:
"""
Sort out \n \r \r\n in full_text column
"""

def replacen(x):
    return x.replace('\n','')
def replacer(x):
    return x.replace('\r','')
def replacern(x):
    return x.replace('\r\n','')

df['full_text'] = df['full_text'].apply(replacer)
df['full_text'] = df['full_text'].apply(replacen)
df['full_text'] = df['full_text'].apply(replacern)


# ignore pink warnings

/var/folders/99/ckqc944s1nl57yz05vhcmt6w0000gn/T/ipykernel_3165/1915096055.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['full_text'] = df['full_text'].apply(replacer)
/var/folders/99/ckqc944s1nl57yz05vhcmt6w0000gn/T/ipykernel_3165/1915096055.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['full_text'] = df['full_text'].apply(replacen)
/var/folders/99/ckqc944s1nl57yz05vhcmt6w0000gn/T/ipykernel_3165/1915096055.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

## Custom One-Hot Encoding for Symbol and Hashtag columns

In [4]:
"""
Categorize the symbol and hastag mentions for the four main culprits of this project:
- Intel (intc)
- Beyond Meat (bynd)
- General Electric (ge)
- Bitcoin (btc)
"""
# Intel
def tickerintc(x):
    if type(x) == list:
        newlist = []
        for item in x:
            newlist.append(item.lower())
        if 'intc' in newlist:
            return 1
        else:
            return 0
        
def hashtag_intc(x):
    if type(x) == list:
        newlist = []
        for item in x:
            newlist.append(item.lower())
        intc = ['intel','intc']
        if intc[0] in newlist or intc[1] in newlist:
            return 1
        else:
            return 0
        
# Beyond Meat
def tickerbynd(x):
    if type(x) == list:
        newlist = []
        for item in x:
            newlist.append(item.lower())
        if 'bynd' in newlist:
            return 1
        else:
            return 0
        
def hashtag_bynd(x):
    if type(x) == list:
        newlist = []
        for item in x:
            newlist.append(item.lower())
        bynd = ['beyondmeat','bynd']
        if bynd[0] in newlist or bynd[1] in newlist:
            return 1
        else:
            return 0
# General Electric
def tickerge(x):
    if type(x) == list:
        newlist = []
        for item in x:
            newlist.append(item.lower())
        if 'ge' in newlist:
            return 1
        else:
            return 0
        
def hashtag_ge(x):
    if type(x) == list:
        newlist = []
        for item in x:
            newlist.append(item.lower())
        ge = ['generalelectric','ge']
        if ge[0] in newlist or ge[1] in newlist:
            return 1
        else:
            return 0
        
# Bitcoin (  >:(   )   
def tickerbtc(x):
    if type(x) == list:
        newlist = []
        for item in x:
            newlist.append(item.lower())
        if 'btc' in newlist:
            return 1
        else:
            return 0
        
def hashtag_btc(x):
    if type(x) == list:
        newlist = []
        for item in x:
            newlist.append(item.lower())
        btc = ['bitcoin','btc']
        if btc[0] in newlist or btc[1] in newlist:
            return 1
        else:
            return 0
##########################################        

## Create our new one-hot encoded ticker columns and fill nans with 0

In [5]:
"""
create our new categorical columns regarding symbols (4x) and hashtags (4x)
"""
        
df['intc_ticker'] = df['symbols'].apply(tickerintc)        
df['bynd_ticker'] = df['symbols'].apply(tickerbynd)        
df['ge_ticker'] = df['symbols'].apply(tickerge)        
df['btc_ticker'] = df['symbols'].apply(tickerbtc)

df['#intc_ticker'] = df['hashtags'].apply(hashtag_intc)        
df['#bynd_ticker'] = df['hashtags'].apply(hashtag_bynd)        
df['#ge_ticker'] = df['hashtags'].apply(hashtag_ge)        
df['#btc_ticker'] = df['hashtags'].apply(hashtag_btc)

"""
fill nan values with zeros in our new columns
"""

df['intc_ticker'].fillna(0.,inplace=True)
df['bynd_ticker'].fillna(0.,inplace=True)
df['ge_ticker'].fillna(0.,inplace=True)
df['btc_ticker'].fillna(0.,inplace=True)

df['#intc_ticker'].fillna(0.,inplace=True)
df['#bynd_ticker'].fillna(0.,inplace=True)
df['#ge_ticker'].fillna(0.,inplace=True)
df['#btc_ticker'].fillna(0.,inplace=True)

# ignore pink warnings

/var/folders/99/ckqc944s1nl57yz05vhcmt6w0000gn/T/ipykernel_3165/3053683736.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['intc_ticker'] = df['symbols'].apply(tickerintc)
/var/folders/99/ckqc944s1nl57yz05vhcmt6w0000gn/T/ipykernel_3165/3053683736.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bynd_ticker'] = df['symbols'].apply(tickerbynd)
/var/folders/99/ckqc944s1nl57yz05vhcmt6w0000gn/T/ipykernel_3165/3053683736.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [6]:
"""
remove nans from ticker colums
"""

df[['intc_ticker','bynd_ticker','ge_ticker','btc_ticker']].fillna(0.,inplace=True)
df[['#intc_ticker','#bynd_ticker','#ge_ticker','#btc_ticker']].fillna(0.,inplace=True)


/Users/ianndelap/.pyenv/versions/3.8.6/envs/market_sentiment/lib/python3.8/site-packages/pandas/core/frame.py:5171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [7]:
"""
fix date time set to day without specific times
"""
df['created_at'] = df['created_at'].dt.date

/var/folders/99/ckqc944s1nl57yz05vhcmt6w0000gn/T/ipykernel_3165/2758922454.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['created_at'] = df['created_at'].dt.date


In [8]:
df

,full_text,lang,datetimestamp,reply_count,retweet_count,quote_count,created_at,favorite_count,symbols,hashtags,intc_ticker,bynd_ticker,ge_ticker,btc_ticker,#intc_ticker,#bynd_ticker,#ge_ticker,#btc_ticker
0,Расцвет человеческого общества зависит от двух...,ru,1583637560,0,0,0,2020-03-08,2,"[btc, alts, wan, strat, mtl, nano, nkn, kmd, z...",NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,I’m waiting for the volatility to settle with ...,en,1583637637,0,0,0,2020-03-08,1,[btc],NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Buy $BTC this is not a fucking drill,en,1583637452,0,0,0,2020-03-08,2,[BTC],NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,"$btc $btcusd nvm, 8,8k target hit",en,1583637470,1,0,0,2020-03-08,1,"[btc, btcusd]",NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,"Mar 8, 2020 03:12:00 UTC The price of $BTC cur...",en,1583637182,0,0,0,2020-03-08,0,[BTC],NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,$LINK TP1 hit 17% Profit so far 🔥👌🏻For 10x gai...,en,1595104240,0,0,0,2020-07-18,0,"[LINK, ETH, UTT, ETHOS, QLC, CND, TNB, LEND, I...","[litecoin, bitcoin, cardano, CRYPTO, ethereum]",0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
6996,Start Trading $BTC On BitMax https://t.co/5nOR...,en,1595104336,0,0,0,2020-07-18,0,[BTC],NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6997,good https://t.co/r6Whwy2bFP 🙋 Bet with $BTC ...,en,1595104294,0,0,0,2020-07-18,0,[BTC],NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6998,$strat / $btc update,en,1595104118,2,1,0,2020-07-18,9,"[strat, btc]",NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [9]:
"""
drop columns we needn't use and check with heatmap for any gaps
"""

df.drop(['favorite_count',
         'lang',
         'datetimestamp',
         'reply_count',
         'retweet_count',
         'quote_count',
         'quote_count',
         'symbols',
         'hashtags'],axis=1,inplace=True)



/Users/ianndelap/.pyenv/versions/3.8.6/envs/market_sentiment/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [10]:
"""
remove links away from full_text
"""

def remove_urls(text):
    text = re.sub(r'https?://\S+', '', text)
    return text

df['full_text'] = df['full_text'].apply(remove_urls)

/var/folders/99/ckqc944s1nl57yz05vhcmt6w0000gn/T/ipykernel_3165/4161786243.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['full_text'] = df['full_text'].apply(remove_urls)


# !! DANGR ZONE !! DO NOT RUN CODE BELOW UNLESS SMALLER DATA

### Make sure data frame does NOT SURPASS 7000 rows!

In [11]:
"""
create a vextor embedding matrix per tweet into a new dataframe variable vector_df
"""

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")
text_result = embed(df['full_text'])
vector_df = pd.DataFrame(text_result.numpy())

2021-09-03 11:15:52.545944: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-09-03 11:15:55.218312: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


# Breathe in, out, and let's check the dataframes we have for comparison

In [13]:
vector_df

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,-0.000039,-0.048648,-0.043249,-0.057029,0.045178,0.028709,-0.047446,-0.059832,0.031897,0.017609,...,-0.026220,0.071336,-0.035513,-0.056909,0.043171,-0.037478,-0.007179,-0.009458,0.013665,0.005449
1,-0.000881,0.006815,-0.104014,-0.102300,-0.089217,0.008585,-0.041062,-0.048969,-0.008564,0.040399,...,-0.053123,-0.057954,0.072325,-0.070757,0.027566,0.002492,0.086193,-0.012549,-0.012987,0.072317
2,0.000568,0.029366,-0.084870,-0.087521,0.033559,0.040141,-0.035663,0.004321,-0.050664,0.001826,...,-0.078234,-0.037736,0.001215,-0.020226,0.067014,0.047181,0.051067,-0.043411,-0.016759,0.061772
3,0.022832,0.036749,-0.079850,-0.088448,-0.063727,0.062514,0.000688,0.024882,0.013593,0.016426,...,-0.024541,-0.051545,0.054564,-0.058412,0.022247,0.010616,0.059010,0.050027,0.039830,0.002944
4,-0.007395,0.053267,-0.086250,-0.085443,0.031137,0.008982,-0.065417,0.012812,0.076530,-0.061635,...,0.029155,-0.070193,0.068606,-0.023226,0.088063,0.013789,0.021891,-0.019742,0.039208,0.028317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,0.000578,-0.013651,-0.082107,-0.079484,0.036194,0.019049,0.010284,-0.044519,0.035540,0.026067,...,0.036901,-0.068532,-0.042788,-0.083297,0.064915,-0.017232,-0.021322,-0.022272,-0.005582,0.028859
6996,0.077788,0.025738,-0.085079,-0.083436,0.012395,0.004047,0.013012,-0.052828,-0.004370,0.043379,...,-0.060502,-0.063820,0.054348,-0.075545,0.050748,0.043078,0.023124,0.002743,-0.023158,0.009475
6997,0.023112,0.066127,-0.096972,-0.093747,-0.029538,0.041717,-0.007597,0.016025,-0.064378,0.068460,...,-0.052450,-0.062375,0.021804,-0.038315,0.035002,0.006192,0.033159,-0.008137,-0.018670,0.070418
6998,0.081348,0.011850,-0.083886,-0.043590,-0.039975,-0.026060,-0.054036,-0.053249,0.059131,0.049116,...,0.029774,-0.047715,0.034008,-0.082192,0.067526,0.023942,0.061132,-0.036305,-0.041049,0.001089


In [14]:
df

,full_text,created_at,intc_ticker,bynd_ticker,ge_ticker,btc_ticker,#intc_ticker,#bynd_ticker,#ge_ticker,#btc_ticker
0,Расцвет человеческого общества зависит от двух...,2020-03-08,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,I’m waiting for the volatility to settle with ...,2020-03-08,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Buy $BTC this is not a fucking drill,2020-03-08,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,"$btc $btcusd nvm, 8,8k target hit",2020-03-08,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,"Mar 8, 2020 03:12:00 UTC The price of $BTC cur...",2020-03-08,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
6995,$LINK TP1 hit 17% Profit so far 🔥👌🏻For 10x gai...,2020-07-18,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
6996,Start Trading $BTC On BitMax,2020-07-18,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6997,good 🙋 Bet with $BTC via ⟶ √,2020-07-18,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6998,$strat / $btc update,2020-07-18,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [15]:
"""
concat the two dataframes one against another to create one single dataframe
"""

df = pd.concat([df,vector_df], axis=1, join="inner")

In [16]:
df.head()

,full_text,created_at,intc_ticker,bynd_ticker,ge_ticker,btc_ticker,#intc_ticker,#bynd_ticker,#ge_ticker,#btc_ticker,...,502,503,504,505,506,507,508,509,510,511
0,Расцвет человеческого общества зависит от двух...,2020-03-08,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,-0.026220,0.071336,-0.035513,-0.056909,0.043171,-0.037478,-0.007179,-0.009458,0.013665,0.005449
1,I’m waiting for the volatility to settle with ...,2020-03-08,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,-0.053123,-0.057954,0.072325,-0.070757,0.027566,0.002492,0.086193,-0.012549,-0.012987,0.072317
2,Buy $BTC this is not a fucking drill,2020-03-08,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,-0.078234,-0.037736,0.001215,-0.020226,0.067014,0.047181,0.051067,-0.043411,-0.016759,0.061772
3,"$btc $btcusd nvm, 8,8k target hit",2020-03-08,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,-0.024541,-0.051545,0.054564,-0.058412,0.022247,0.010616,0.059010,0.050027,0.039830,0.002944
4,"Mar 8, 2020 03:12:00 UTC The price of $BTC cur...",2020-03-08,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.029155,-0.070193,0.068606,-0.023226,0.088063,0.013789,0.021891,-0.019742,0.039208,0.028317


## Choose conditional dataframes for tickers and hashtags

In [17]:
"""
select the dataframes for when the symbol tickers have a hit
"""

intc_ticker_df = df[df['intc_ticker']==1.]
bynd_ticker_df = df[df['bynd_ticker']==1.]
ge_ticker_df = df[df['ge_ticker']==1.]
btc_ticker_df = df[df['btc_ticker']==1.]

"""
select the dataframes for when the hashtag tickers have a hit
"""

intc_hash_ticker_df = df[df['#intc_ticker']==1.]
bynd_hash_ticker_df = df[df['#bynd_ticker']==1.]
ge_hash_ticker_df = df[df['#ge_ticker']==1.]
btc_hash_ticker_df = df[df['#btc_ticker']==1.]

In [18]:
intc_ticker_mean_df = intc_ticker_df.groupby('created_at').mean()
bynd_ticker_mean_df = bynd_ticker_df.groupby('created_at').mean()
ge_ticker_mean_df = ge_ticker_df.groupby('created_at').mean()
btc_ticker_mean_df = btc_ticker_df.groupby('created_at').mean()
                   
intc_hash_ticker_mean_df = intc_hash_ticker_df.groupby('created_at').mean()
bynd_hash_ticker_mean_df = bynd_hash_ticker_df.groupby('created_at').mean()
ge_hash_ticker_mean_df = ge_hash_ticker_df.groupby('created_at').mean()
btc_hash_ticker_mean_df = btc_hash_ticker_df.groupby('created_at').mean()

In [19]:
"""
convert created_at index from object to datetime 
"""
# symbol ticker dfs
intc_ticker_mean_df.index = pd.to_datetime(intc_ticker_mean_df.index)
bynd_ticker_mean_df.index = pd.to_datetime(bynd_ticker_mean_df.index)
ge_ticker_mean_df.index = pd.to_datetime(ge_ticker_mean_df.index)
btc_ticker_mean_df.index = pd.to_datetime(btc_ticker_mean_df.index)
# hashtag ticker dfs
intc_hash_ticker_mean_df.index = pd.to_datetime(intc_hash_ticker_mean_df.index)
bynd_hash_ticker_mean_df.index = pd.to_datetime(bynd_hash_ticker_mean_df.index)
ge_hash_ticker_mean_df.index = pd.to_datetime(ge_hash_ticker_mean_df.index)
btc_hash_ticker_mean_df.index = pd.to_datetime(btc_hash_ticker_mean_df.index)

# Import Target Dataframe

In [20]:
import yfinance as yf
tickers = yf.download(tickers = 'INTC BYND GE BTC', interval='1d',start="2018-01-01", end="2021-08-01")

[*********************100%***********************]  4 of 4 completed


# Encode the output to give us our target series

In [21]:
"""
Convert tickers to 1 and 0 as target output series
"""

INTCclose = tickers['Close']['INTC']
BYNDclose = tickers['Close']['BYND']
GEclose = tickers['Close']['GE']
BTCclose = tickers['Close']['BTC']

def onezero(x):
    if x>0:
        return 1
    else:
        return 0

intc_target_df = pd.Series(INTCclose).diff().apply(onezero)
bynd_target_df = pd.Series(BYNDclose).diff().apply(onezero)
ge_target_df = pd.Series(GEclose).diff().apply(onezero)
btc_target_df = pd.Series(BTCclose).diff().apply(onezero)

# Merge our target series to the data frame matching one of the 4 market culprits we're looking at

In [22]:
"""
Merge ticker feature selected aggredated dataframe with target target dataframe created
"""

INTC_df = pd.merge(intc_ticker_mean_df,intc_target_df,left_index=True,right_index=True)

In [23]:
"""
Merge ticker feature selected aggredated dataframe with target target dataframe created
"""

BYND_df = pd.merge(bynd_ticker_mean_df,bynd_target_df,left_index=True,right_index=True)

In [24]:
"""
Merge ticker feature selected aggredated dataframe with target target dataframe created
"""

GE_df = pd.merge(ge_ticker_mean_df,ge_target_df,left_index=True,right_index=True)

In [25]:
"""
Merge ticker feature selected aggredated dataframe with target target dataframe created
"""

BTC_df = pd.merge(btc_ticker_mean_df,btc_target_df,left_index=True,right_index=True)

# We not have four data frames with a target and aggregated vectorization. The dataframes are named:
### - INTC_df
### - BYND_df
### - GE_df
### - BTC

# model

In [26]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor


In [32]:
BTC_df

,intc_ticker,bynd_ticker,ge_ticker,btc_ticker,#intc_ticker,#bynd_ticker,#ge_ticker,#btc_ticker,0,1,...,503,504,505,506,507,508,509,510,511,BTC
2018-05-09,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.336175,0.016199,0.001838,...,-0.048503,0.001523,-0.068453,0.046241,0.014057,-0.015222,-0.018087,-0.000527,0.035419,0
2018-09-18,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.268116,0.015754,0.009827,...,-0.053478,0.001926,-0.071009,0.050112,0.019785,-0.020967,-0.015656,0.000594,0.025667,0
2018-09-19,0.000000,0.000000,0.001408,1.0,0.0,0.0,0.0,0.347887,0.015489,0.007858,...,-0.054538,-0.004882,-0.068595,0.046215,0.018383,-0.019250,-0.017848,-0.003772,0.027624,0
2019-02-01,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.000000,0.088351,0.054780,...,-0.063440,0.076560,-0.079728,0.071660,-0.005419,0.050477,0.070211,0.042883,0.082415,0
2019-06-14,0.001698,0.005093,0.005093,1.0,0.0,0.0,0.0,0.280136,0.011876,0.011259,...,-0.049814,0.005452,-0.064782,0.046943,0.016129,-0.013485,-0.019587,-0.000494,0.035247,0
2019-10-25,0.000000,0.001340,0.000000,1.0,0.0,0.0,0.0,0.345845,0.010280,0.018830,...,-0.047919,0.013401,-0.061197,0.042689,0.018223,-0.003999,-0.018087,0.003252,0.040256,0
2020-02-28,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.000000,-0.022626,-0.062570,...,-0.017612,0.029001,-0.059762,0.038988,0.035195,0.052776,-0.076614,0.023414,0.054135,0
2020-03-02,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.000000,0.074202,-0.077948,...,0.017653,0.062903,-0.091912,0.030905,0.013141,0.027823,-0.012890,-0.041475,0.065177,0
2020-03-03,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,1.000000,0.039671,-0.070008,...,-0.074560,-0.022291,-0.075621,0.069339,0.049339,-0.059214,0.048741,-0.012951,-0.000595,0
2020-03-04,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.000000,0.061630,-0.054590,...,-0.035481,-0.013759,-0.077884,0.059235,0.021716,0.026960,0.012880,0.006999,0.038281,0


In [47]:
X = BTC_df.drop(columns = ['intc_ticker','ge_ticker','#intc_ticker','bynd_ticker','#bynd_ticker', '#ge_ticker','#btc_ticker','BTC'])
y= BTC_df['BTC']

In [94]:
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.ensemble import AdaBoostClassifier

In [95]:
model = AdaBoostClassifier()

In [96]:
pipe_baseline = Pipeline([('nameofmodel', model)])

In [98]:
tree = DecisionTreeRegressor()
cv_results = cross_validate(pipe_baseline, X, y, scoring = 'accuracy' , cv=5)

#cv_results['test_score'].std()
cv_results['test_score'].mean()

/Users/ianndelap/.pyenv/versions/3.8.6/envs/market_sentiment/lib/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


0.5666666666666667

In [77]:
#change scoring:
#Creat a function eg: 
def score_func(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

rsmle = make_scorer(score_func)


In [ ]:
grid_search.best_params_

In [99]:
grid_search = GridSearchCV(model, 
    param_grid={'nameofmodel__n_estimators':[3,5,10],
                'nameofmodel__learning_rate':[0,5,0,7],
        
    },
    cv=5,
    scoring='accuracy')


NameError: name 'GridSearchCV' is not defined